This code uses the ratiometric indicator Bodipy 581/591 C11 to look at how lipid peroxidation changes with the knockdown of ADAMTS19 and TAX1BP1 based on the observation that during oxidation, the flourescent properties of the indicator shift from red to green, allowing lipid peroxidation to be measured ratiometrically.

Import Libraries

In [ ]:
import czifile
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
from skimage.color import label2rgb, rgb2hsv
from skimage.filters import gaussian, sobel
from skimage.measure import label, regionprops
from skimage.morphology import binary_erosion, binary_dilation, disk, local_maxima
from scipy.ndimage import binary_fill_holes
from skimage.feature import blob_log
from skimage.color import rgb2gray
from skimage import exposure
from skimage.io import imshow
from skimage.draw import circle_perimeter
from matplotlib.colors import LogNorm
import scipy.ndimage as ndi
import pandas as pd
import os

Define Sub Function

In [ ]:
def process_file(image_path, basename):
    image = czifile.imread(image_path)
    image_squeezed = np.squeeze(image)
    red_intensities = []
    green_intensities = []
    last_timepoint = image_squeezed[-1, 0, :,:]
    mask = gaussian(last_timepoint) > 0.02 #Don't include blank space in between cells by segmenting out cells from brightest image, which is the last one
    for slice in range(0,image_squeezed.shape[0]):
        red_channel = image_squeezed[slice,0,:,:]
        mean_intensity_red = np.mean(red_channel[mask > 0])
        red_intensities.append(mean_intensity_red)
        green_channel = image_squeezed[slice,1,:,:]
        mean_intensity_green = np.mean(green_channel[mask > 0])
        green_intensities.append(mean_intensity_green)
    df = pd.DataFrame({
        "Filename": [basename] * len(red_intensities),
        "MFI_Red": red_intensities,
        "MFI_Green": green_intensities,
        "Green_Red_Ratio": [x/y for x,y in zip(green_intensities,red_intensities)]
        })
    
    return df

Main Function

In [ ]:
all_data = []

folder_path = r"test_images"
for well_image in os.listdir(folder_path):
    if well_image.lower().endswith(".czi"):  # Filter for CZI files
        well_image_path = os.path.join(folder_path, well_image)
        well_image_base_name = os.path.basename(well_image)[:-4]
        df = process_file(well_image_path, well_image_base_name)
        all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined DataFrame to a single CSV
combined_csv_path = os.path.join("BODIPY.xlsx")
combined_df.to_excel(combined_csv_path, index=False)